In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [20]:
ano = [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
List_AVG = ['rating','minutes','offsides','shots_total','shots_on','goals_total','goals_conceded','assists','saves','passes_total','passes_key','passes_accuracy','tackles_total','tackles_blocks','tackles_interceptions','duels_total','duels_won','dribbles_attempts','dribbles_success','dribbles_past','fouls_drawn','fouls_committed','cards_yellow','cards_red','penalty_won','penalty_committed','penalty_scored','penalty_missed','penalty_saved']

In [8]:
# Lista de anos do campeonato brasileiro
anos = [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]

# Loop único sobre cada ano
for ano in anos:
    # Carregar os DataFrames
    globals()[f"df_escalacao_{ano}"] = pd.read_csv(f"../Data/Camp_Brasileiro_{ano}_escalacao.csv")
    globals()[f"df_team_stats_{ano}"] = pd.read_csv(f"../Data_Lake/Camp_Brasileiro/{ano}/Camp_Brasileiro_{ano}_team_stats_final.csv")
    globals()[f"df_stats_{ano}"] = pd.read_csv(f"../Data_Lake/Camp_Brasileiro/{ano}/Camp_Brasileiro_{ano}_players_stats_final.csv")
    globals()[f"df_Games_{ano}"] = pd.read_csv(f"../Data_Lake/Camp_Brasileiro/{ano}/Camp_Brasileiro_{ano}_Games_final.csv")
    
    # Criar a lista de fixture_id diretamente no mesmo loop
    globals()[f"lista_games_{ano}"] = globals()[f"df_stats_{ano}"]['fixture_id'].unique().tolist()

In [ ]:
def calcular_media_stats(df, fixture_x, jogos_anteriores=5):
    # Lista de jogadores que participaram do fixture_x
    jogadores_em_x = df[df['fixture_id'] == fixture_x]['player_id'].unique()
    
    # Filtra os jogos anteriores dos jogadores de x
    jogos_anteriores = df[(df['player_id'].isin(jogadores_em_x)) & (df['fixture_id'] < fixture_x)]
    
    # Ordena os jogos por jogador e fixture_id (cronologicamente)
    jogos_ordenados = jogos_anteriores.sort_values(['player_id', 'fixture_id'])
    
    # Seleciona os três últimos jogos de cada jogador
    ultimos_tres = jogos_ordenados.groupby('player_id').tail(jogos_anteriores)           #Quantos jogos vou pegar 
    
    # Calcula a média das estatísticas
    media_estatisticas = ultimos_tres.groupby(['player_id','team_id'])[List_AVG].mean().reset_index()
    
    # Garante todos os jogadores de x, mesmo sem jogos anteriores
    resultado_final = pd.DataFrame({'player_id': jogadores_em_x})
    resultado_final = resultado_final.merge(media_estatisticas, on='player_id', how='left')
    
    return resultado_final

In [22]:
import pandas as pd
pd.set_option('display.max_columns', None)

# Lista de colunas para cálculo da média (DEFINA ISSO!)
List_AVG = ['gols', 'assistencias', 'finalizacoes_no_gol', 'passes_certos']

# Dicionários para armazenar dados
df_escalacao_dict = {}
df_team_stats_dict = {}
df_stats_dict = {}
df_Games_dict = {}
lista_games_dict = {}

# Loop sobre os anos
anos = [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
for ano in anos:
    try:
        df_escalacao_dict[ano] = pd.read_csv(f"../Data/Camp_Brasileiro_{ano}_escalacao.csv")
        df_team_stats_dict[ano] = pd.read_csv(f"../Data_Lake/Camp_Brasileiro/{ano}/Camp_Brasileiro_{ano}_team_stats_final.csv")
        df_stats_dict[ano] = pd.read_csv(f"../Data_Lake/Camp_Brasileiro/{ano}/Camp_Brasileiro_{ano}_players_stats_final.csv")
        df_Games_dict[ano] = pd.read_csv(f"../Data_Lake/Camp_Brasileiro/{ano}/Camp_Brasileiro_{ano}_Games_final.csv")
        lista_games_dict[ano] = df_stats_dict[ano]['fixture_id'].unique().tolist()
    except FileNotFoundError as e:
        print(f"Erro ao carregar dados de {ano}: {e}")

# Função com parâmetros flexíveis e tratamento de NaN
def calcular_media_stats(df, fixture_x, jogos_anteriores=5):
    jogadores_em_x = df[df['fixture_id'] == fixture_x]['player_id'].unique()
    jogos_anteriores = df[(df['player_id'].isin(jogadores_em_x)) & (df['fixture_id'] < fixture_x)]
    jogos_ordenados = jogos_anteriores.sort_values(['player_id', 'fixture_id'])
    ultimos_tres = jogos_ordenados.groupby('player_id').tail(jogos_anteriores)
    
    if ultimos_tres.empty:
        return pd.DataFrame({'player_id': jogadores_em_x}).fillna(0)
    
    media_estatisticas = ultimos_tres.groupby(['player_id', 'team_id'])[List_AVG].mean().reset_index()
    resultado_final = pd.DataFrame({'player_id': jogadores_em_x})
    resultado_final = resultado_final.merge(media_estatisticas, on='player_id', how='left').fillna(0)
    
    return resultado_final

In [24]:
def calcular_media_stats(df, fixture_x, jogos_anteriores=5):
    """
    Calcula a média das estatísticas dos jogadores nos jogos anteriores a um fixture específico.
    
    Parâmetros:
        df (pd.DataFrame): DataFrame com estatísticas dos jogadores.
        fixture_x (int): ID do fixture de referência.
        jogos_anteriores (int): Número de jogos anteriores a considerar (padrão=5).
    
    Retorna:
        pd.DataFrame: Médias das estatísticas por jogador.
    """
    # Filtra os jogadores que participaram do fixture_x
    jogadores_em_x = df[df['fixture_id'] == fixture_x]['player_id'].unique()
    
    # Filtra os jogos anteriores dos jogadores de x
    jogos_anteriores = df[(df['player_id'].isin(jogadores_em_x)) & (df['fixture_id'] < fixture_x)]
    
    # Se não houver jogos anteriores, retorna um DataFrame vazio
    if jogos_anteriores.empty:
        print(f"Nenhum jogo anterior encontrado para fixture {fixture_x}")
        return pd.DataFrame(columns=['player_id', 'team_id'] + List_AVG + ['fixture_id'])
    
    # Ordena os jogos por jogador e fixture_id (cronologicamente)
    jogos_ordenados = jogos_anteriores.sort_values(['player_id', 'fixture_id'])
    
    # Seleciona os últimos jogos de cada jogador
    ultimos_tres = jogos_ordenados.groupby('player_id').tail(jogos_anteriores)
    
    # Calcula a média das estatísticas
    media_estatisticas = ultimos_tres.groupby(['player_id', 'team_id'])[List_AVG].mean().reset_index()
    
    # Garante todos os jogadores de x, mesmo sem jogos anteriores
    resultado_final = pd.DataFrame({'player_id': jogadores_em_x})
    resultado_final = resultado_final.merge(media_estatisticas, on='player_id', how='left').fillna(0)
    
    # Adiciona o fixture_id ao resultado
    resultado_final['fixture_id'] = fixture_x
    
    return resultado_final

In [27]:
calcular_media_stats(df_stats_dict[2024],1180727,jogos_anteriores=5)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
# Lista para armazenar os DataFrames de cada fixture
lista_dfs = []

# Iterar sobre cada fixture_id na lista_games_2024
for fixture_x in lista_games_2024:
    # Calcular a média dos últimos três jogos para o fixture atual
    df_medias = calcular_media_stats(df_stats, fixture_x)
    
    # Adicionar a coluna 'fixture_id' ao DataFrame resultante (para identificar o jogo)
    df_medias['fixture_id'] = fixture_x
    
    # Adicionar à lista
    lista_dfs.append(df_medias)

# Concatenar todos os DataFrames da lista em um único df_features
df_features = pd.concat(lista_dfs, ignore_index=True)